In [2]:
from ctypes import windll
windll.shcore.SetProcessDpiAwareness(1)
import tkinter as tk
import pickle
import sys
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

def process(d1,d2,w1,w2):
    #由你的电脑的屏幕大小来决定
    def get_screen_size():
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        root.destroy()
        return screen_width, screen_height
    
    #实验数据产生的设备的屏幕大小
    screen_wide,screen_high=get_screen_size()

    print(screen_wide,screen_high)
    
    x_edge1=screen_wide/2-d1/2-w1/2
    x_edge2=screen_wide/2-d1/2+w1/2
    x_edge3=screen_wide/2+d1/2-w1/2
    x_edge4=screen_wide/2+d1/2+w1/2

    

    def read_data(d1,d2,w1,w2):
        '''
        通过D,和W的大小来区分和读取数据，并返回四个数据结果。
        '''
        name=sys.path[0]+'\data\\'+'D='+str(d1)+'-'+str(d2)+'_W='+str(w1)+'-'+str(w2)+".bin"

        with open(name,'rb') as file:
            move_datas=pickle.load(file)
        
        return move_datas

    mouse_move=read_data(d1,d2,w1,w2)

    
    for i in range(len(mouse_move)):
        pass
        #print(np.shape(mouse_move[i]))

    def clean(mouse_move):
        new1=[]
        for i in range(len(mouse_move)):
            new2=[]
            t=-1
            x=-1
            y=-1
            for j in range(len(mouse_move[i])):
                
                if t==mouse_move[i][j][0] and x==mouse_move[i][j][1] and y==mouse_move[i][j][2]:
                    t=mouse_move[i][j][0]
                    x=mouse_move[i][j][1]
                    y=mouse_move[i][j][2]
                    continue
                else:
                    t=mouse_move[i][j][0]
                    x=mouse_move[i][j][1]
                    y=mouse_move[i][j][2]
                    new2.append(mouse_move[i][j])
            new1.append(new2)
        return new1

    mouse_move=clean(mouse_move)

    def average(mouse_move):
        new=[]
        for i in range(len(mouse_move)):
            new1=[]
            for j in range(len(mouse_move[i])-1):
                t=mouse_move[i][j][0]
                if mouse_move[i][j][0]==mouse_move[i][j+1][0]:
                    r=1
                    for k in range(j+1,len(mouse_move[i])-1):
                        if mouse_move[i][j][0]==mouse_move[i][k][0]:
                            r+=1
                            #print(k)
                    #print('对于',mouse_move[i][j][0],'有',r,'个连续重复值')
                    
                    t0=mouse_move[i][j-1][0]
                    t1=mouse_move[i][j+r][0]
                    m=(t1-t0)/(r+1)
                    t=t0+m*2
                    
                new1.append([t,mouse_move[i][j][1],mouse_move[i][j][2]])
            new.append(new1)    
        
        for i in range(len(new)):
            for j in range(len(new[i])-1):
                if new[i][j][0]==new[i][j+1][0]:
                    print('存在前后相同点：',i,j)
        
        return new

    mouse_move=average(mouse_move)
    
    t=[]
    for i in range(len(mouse_move)):
        #print(np.shape(mouse_move[i]))
        #print(mouse_move[i][-1][0]-mouse_move[i][0][0])
        t.append(mouse_move[i][-1][0]-mouse_move[i][0][0])

    print('平均时间为',np.mean(t))
    
    ind=[]
    print(x_edge1,x_edge2,x_edge3,x_edge4)
    for i in range(len(mouse_move)):
        x1=0
        x2=0
        for j in range(len(mouse_move[i])):
            if mouse_move[i][j][1]>x_edge2:
                x1=j
                break
        for k in range(len(mouse_move[i])):
            if mouse_move[i][k][1]>x_edge3:
                x2=k
                break
        ind.append([x1,x2])
    print(ind)
    
    times=[]
    all_t1=[]
    all_t2=[]
    all_t3=[]
    all_t4=[]

    for i in range(len(ind)):
        t0=mouse_move[i][0][0]
        t1=mouse_move[i][ind[i][0]][0]-t0
        t2=mouse_move[i][ind[i][1]][0]-t0
        t3=mouse_move[i][-1][0]-t0
        
        d_t1=t1
        d_t2=t2-t1
        d_t3=t3-t2
        
        all_t1.append(d_t1)
        all_t2.append(d_t2)
        all_t3.append(d_t3)
        all_t4.append(t3)
        
        #print(t1,t2,t3)
        times.append([d_t1,d_t2,d_t3,t3])

    def mean_without_max_min(arr):
        
        # 找到最大值和最小值
        max_value = np.max(arr)
        min_value = np.min(arr)

        # 使用布尔数组作为索引，找到不等于最大值和最小值的元素
        arr_without_extremes = (np.sum(arr)-max_value-min_value)/(len(arr)-2)

        return arr_without_extremes
    #mean_t1,mean_t2,mean_t3,mean_t4=mean_without_max_min(all_t1),mean_without_max_min(all_t2),mean_without_max_min(all_t3),mean_without_max_min(all_t4)
    mean_t1,mean_t2,mean_t3,mean_t4=np.mean(all_t1),np.mean(all_t2),np.mean(all_t3),np.mean(all_t4)#没有去掉最大最小值
    times.append([mean_t1,mean_t2,mean_t3,mean_t4])

    return times

import os
folder_path=sys.path[0]+'\image'
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print(f"Folder '{folder_path}' created successfully.")



d=[2500,2000,1500,1000,500,250,125]
w=[80,60,40,20]





for i in range(len(d)):
    times=[]
    folder_path=sys.path[0]+'\image\D_'+str(d[i])
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    for j in range(len(w)):
        times.append(process(d[i],d[i],w[j],w[j]))
    
    colors = [
    ('red', '#FF0000'),
    ('orange', '#FFA500'),
    ('yellow', '#FFFF00'),
    ('green', '#00FF00'),
    ('blue', '#0000FF'),
    ('cyan', '#00FFFF'),
    ('purple', '#800080'),
    ('black', '#000000'),

    ('lime', '#00FF00'),

    ('magenta', '#FF00FF'),

    ('darkred', '#8B0000'),
    ('darkgreen', '#006400'),
    ('darkblue', '#00008B'),
    ('darkgoldenrod', '#B8860B'),

    ('hotpink', '#FF69B4'),
    ('teal', '#008080'),
    ('silver', '#C0C0C0'),
    ('saddlebrown', '#8B4513'),

    ]

    plt.figure(dpi=200)
    X=[1,2,3,4]
    for k in range(len(times)):
        #plt.axis('equal')
        # 绘制原始数据和拟合曲线
        plt.plot(X,times[k][-1],color=colors[k][0])
        #for j in range(len(times[k][-1])):
            #plt.text(X[j], times[k][-1][j], str(times[k][-1][j]), ha='center', va='bottom')
    plt.xlabel('time')
    plt.ylabel('value')
    plt.title('D='+str(d[i]))
    #plt.legend()
    plt.grid(True)
    #plt.show()
    # 保存图像为PNG格式
    plt.savefig(folder_path+'\\D='+str(d[i])+'.png')

    # 可以选择保存为其他格式，比如JPEG格式
    # plt.savefig('example.jpg')

    # 保存后需要关闭绘图窗口
    plt.close()
    
for i in range(len(w)):
    times=[]
    folder_path=sys.path[0]+'\image\W_'+str(w[i])
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    for j in range(len(d)):
        times.append(process(d[j],d[j],w[i],w[i]))
    
    colors = [
    ('red', '#FF0000'),
    ('orange', '#FFA500'),
    ('yellow', '#FFFF00'),
    ('green', '#00FF00'),
    ('blue', '#0000FF'),
    ('cyan', '#00FFFF'),
    ('purple', '#800080'),
    ('black', '#000000'),

    ('lime', '#00FF00'),

    ('magenta', '#FF00FF'),

    ('darkred', '#8B0000'),
    ('darkgreen', '#006400'),
    ('darkblue', '#00008B'),
    ('darkgoldenrod', '#B8860B'),

    ('hotpink', '#FF69B4'),
    ('teal', '#008080'),
    ('silver', '#C0C0C0'),
    ('saddlebrown', '#8B4513'),

    ]

    plt.figure(dpi=200)
    X=[1,2,3,4]
    for k in range(len(times)):
        #plt.axis('equal')
        # 绘制原始数据和拟合曲线
        plt.plot(X,times[k][-1],color=colors[k][0])
        #for j in range(len(times[k][-1])):
            #plt.text(X[j], times[k][-1][j], str(times[k][-1][j]), ha='center', va='bottom')
    plt.xlabel('time')
    plt.ylabel('value')
    plt.title('W='+str(w[i]))
    #plt.legend()
    plt.grid(True)
    #plt.show()
    
    # 保存图像为PNG格式
    plt.savefig(folder_path+'\\W='+str(w[i])+'.png')

    # 可以选择保存为其他格式，比如JPEG格式
    # plt.savefig('example.jpg')

    # 保存后需要关闭绘图窗口
    plt.close()

2560 1600
平均时间为 1.0194783210754395
-10.0 70.0 2490.0 2570.0
[[148, 694], [162, 1732], [193, 1860], [187, 1923], [169, 1726], [149, 2076], [151, 1758], [263, 1388], [120, 1575], [165, 1553], [157, 1537], [212, 1639], [155, 964], [121, 1927], [149, 1503], [151, 1513]]
2560 1600
平均时间为 0.9805422425270081
0.0 60.0 2500.0 2560.0
[[244, 710], [132, 762], [137, 743], [126, 1680], [152, 823], [149, 1692], [129, 946], [171, 1882], [151, 1418], [161, 1465], [170, 1580], [179, 1820], [157, 1997], [143, 1791], [160, 742], [143, 1112]]
2560 1600
平均时间为 1.1432851999998093
10.0 50.0 2510.0 2550.0
[[158, 1484], [131, 2066], [117, 1881], [255, 1764], [142, 1540], [140, 1990], [266, 1936], [122, 855], [115, 2068], [126, 1895], [140, 1337], [121, 1834], [141, 1448], [139, 1813], [115, 1864], [153, 1836]]
2560 1600
平均时间为 1.144937738776207
20.0 40.0 2520.0 2540.0
[[155, 1891], [142, 679], [182, 1893], [131, 1533], [108, 865], [108, 1774], [181, 1048], [146, 2374], [174, 843], [157, 2011], [129, 774], [177, 2